In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import sys, os
sys.path.append(os.path.expanduser('~/Desktop'))
from myAnimalShelter import DatabaseLayer


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser1"
password = "SNHU1234"
shelter = DatabaseLayer(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)


dd_Water = {"animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte": 26, "$lte":156},
            "outcome_type":{"$nin":["Euthanasia"]}}

dd_Mountain = {"animal_type":"Dog",
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
            "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 26, "$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}}

dd_Disater = {"animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
            "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 20, "$lte": 300},
            "outcome_type":{"$nin":["Euthanasia"]}}

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image = 'Grazioso.png'
encoded_image = base64.b64encode(open(image, 'rb').read())

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Tam Phan SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'10%','width':'10%'}
        ),
    html.Div(
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # Dropdown Filtering Option
        dcc.Dropdown(id='dropdown_filter',
                      options=[
                          {'label':'Water Rescue', 'value':'Water'},
                          {'label':'Mountain Rescue', 'value':'Mountain'},
                          {'label':'Disater Rescue', 'value':'Disater'},
                          {'label':'All', 'value':'All'},
                      ])
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        filter_action="native", 
        sort_action="native", 
        sort_mode="multi",
        selected_columns=[],
        column_selectable=False,
        page_action="native",
        page_current = 0,
        page_size=10, 

    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#Dashboard for Dropdown Filter
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('dropdown_filter', 'value')])
              
def update_dashboard(dropdown_filter):
    if dropdown_filter == 'Water':
        df = pd.DataFrame(list(shelter.read(dd_Water)))
    elif dropdown_filter == 'Mountain':
        df = pd.DataFrame(list(shelter.read(dd_Mountain)))
    elif dropdown_filter == 'Disater':
        df = pd.DataFrame(list(shelter.read(dd_Disater)))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    data=df.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    return (data, columns)
              
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Pie Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def generate_pie(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
        figure = px.pie(dff, names="breed", title='DOGS BREED'))
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):

    # Extract the selected row from the Panda DataFrame:
    dff = pd.DataFrame.from_dict(viewData)
    
    # As long as you enforce single row selection, the list can
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]
 
    # Return the appropriate part of the map, centered on the latitude and longitude
    # of the selected row:
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for
                # the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                
                # So, add a "dl.Marker" here, whose position is set
                # to the latitude (row 13) and longitude (row 14) of the 
                # selected row and the the Popup will use the name from row 9:
                dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                          children=[
                              dl.Tooltip(dff.iloc[row,4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row,9])
                              ])
                          ])
            ])
    ]
                  

app.run_server(debug=True)

Dash app running on http://127.0.0.1:10776/
